In [53]:
# import librairies
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime as dt
import matplotlib.pyplot 

#### 1. Créer la table person dans la base de données choisie.

In [54]:
# création de la database HDH

connection = sqlite3.connect("hdh.db")
cursor = connection.cursor()

# création de la table person dans la BDD HDH

cursor.execute('''
		CREATE TABLE IF NOT EXISTS person (
			person_id int primary key,
			person_source_value text,
			gender_concept_id int,
            year_of_birth int
			)
               ''')

connection.commit()

#### 2. Ouvrir les tables T_MCOAAC et T_MCOAAB, et enlever les doublons et les valeurs manquantes. Calculer le nombre de lignes pour chaque table, la distribution homme/femme de la table T_MCOAAB.

In [55]:
# importation des 2 fichiers csv en dataframes

mcoaab_df = pd.read_csv(r'T_MCOaaB.csv', sep=',')
mcoaac_df = pd.read_csv(r'T_MCOaaC.csv', sep='|')

In [56]:
# vérifications du bon import des fichiers

mcoaab_df.head()
mcoaac_df.head()

,NIR_ANO_17,RSA_NUM,ETA_NUM,COH_NAI_RET,COH_SEX_RET,DAT_RET,ENT_DAT,ETA_NUM_RET,EXE_SOI_AMD,EXE_SOI_AMF,...,SEJ_MER_RET,SEJ_NUM,SEJ_RET,SEX_RET,SOR_ANN,SOR_DAT,SOR_MOI,VID_HOSP_FOR,ENT_AM,SOR_AM
0,fiosftusfZACfuNhE,6766543446,250010162,True,False,c,5122013,True,198104,198104,...,True,31556,False,True,2014,1012014,6,418,199211,199211
1,fiosftusfZACfuNhE,1778813122,470015009,True,True,v,10032016,True,201310,201310,...,False,97756,False,True,1998,15042016,6,509,197208,197208
2,fiosftusfZACfuNhE,6079331430,30005540,False,True,Q,8021998,False,201106,201106,...,False,81628,False,False,2005,11031998,1,785,197205,197206
3,fiosftusfZACfuNhE,8205197405,750039182,False,False,V,18072010,False,201710,201711,...,False,69840,True,True,2001,18082010,2,410,201302,201302
4,fiosftusfZACfuNhE,8501373207,920003399,True,True,u,10111986,True,201208,201208,...,True,65673,False,False,1996,26111986,9,237,201211,201212


In [57]:
# supression des doublons en prenant en compte le double critères d'unicité ETA_NUM + RSA_NUM

mcoaab_df2 = mcoaab_df.drop_duplicates(subset=['ETA_NUM','RSA_NUM'])
mcoaac_df2 = mcoaac_df.drop_duplicates(subset=['ETA_NUM','RSA_NUM'])


In [58]:
# vérifications de l'existence de valeurs manquantes dans toutes les colonnes

mcoaab_df2.isnull().sum()
mcoaac_df2.isnull().sum()

NIR_ANO_17         0
RSA_NUM            0
ETA_NUM            0
COH_NAI_RET        0
COH_SEX_RET        0
DAT_RET            0
ENT_DAT            0
ETA_NUM_RET        0
EXE_SOI_AMD        0
EXE_SOI_AMF        0
EXE_SOI_DTD        0
EXE_SOI_DTF        0
FHO_RET            0
FOR_NUM            0
HOS_NN_MAM         0
HOS_NNE_RET        0
HOS_ORG_RET        0
HOS_PLO            0
NAI_RET            0
NIR_ANO_MAM        0
NIR_RET            0
NUM_DAT_AT         0
NUM_DAT_AT_RET     0
ORG_CPL_NUM        0
ORG_CPL_NUM_RET    0
PMS_RET            0
RNG_BEN            0
RNG_BEN_RET        0
RNG_NAI            0
RNG_NAI_RET        0
SEJ_MER_RET        0
SEJ_NUM            0
SEJ_RET            0
SEX_RET            0
SOR_ANN            0
SOR_DAT            0
SOR_MOI            0
VID_HOSP_FOR       0
ENT_AM             0
SOR_AM             0
dtype: int64

In [59]:
# Suppression valeurs manquantes

mcoaab_df2_copy = mcoaab_df2.copy()
mcoaab_df2_copy.dropna(subset=["ETA_NUM","RSA_NUM"], inplace=True)

mcoaac_df2_copy = mcoaac_df2.copy()
mcoaac_df2_copy.dropna(subset=["ETA_NUM", "RSA_NUM"], inplace=True)

In [60]:
# calcul nombre de lignes pour chaque table
vol1 = len(mcoaab_df2_copy)
vol2 = len(mcoaac_df2_copy)

f"Nombre de lignes de la table T_MCOaaB : {vol1}"
f"Nombre de lignes de la table T_MCOaaC : {vol2}"


'Nombre de lignes de la table T_MCOaaC : 514'

In [61]:
# distribution homme/femme de la table T_MCOaaB

cs = mcoaab_df2_copy["COD_SEX"][mcoaab_df2_copy["COD_SEX"].isin([1,2])]

volume = cs.value_counts()
ratio = cs.value_counts(normalize=True)
pourcentage = cs.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'VOLUME': volume, 'RATIO': ratio, 'POURCENTAGE': pourcentage})

,VOLUME,RATIO,POURCENTAGE
2,190,0.535211,53.5%
1,165,0.464789,46.5%


#### 3. Remplir la colonne **person_source_value** de la table *person*.